# DATOS DE TRANCE DE LA API DE SPOTIFY

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
sp_clave = os.getenv("irene_id_sp")
sp_contraseña = os.getenv("irene_secret_sp")

In [5]:
#Autentificación:
auth_manager = sp = SpotifyClientCredentials(client_id=sp_clave,
                                             client_secret=sp_contraseña)

sp = spotipy.Spotify(auth_manager=auth_manager)

In [6]:
# buscamos con SEARCH - SEARCH FOR ITEM y aquí vemos los parámetros q son cesarios:

# 500 canciones por género y año ===> 250
# offset nos hace "páginas" en base al índice de los elementos
# limt = 50 y offset = 0 y luego 50, 100, 150, 200. etc hasta llegar al 500
# pensar una variable xa q nos lo haga solo
#for i in range(0, 5001, 50):
#   print(i)
#for i in range(2020, 2025):  # escogemos 5 años entre 2019-2023
#    q= f"genre:rock, year:{i}"
#    print(q)

# include_external no nos interesa

# vamos a Spotipy y vemos cómo se solicita desde aqui
# tenemos SEARCH:
# sp.search(q="", type="track", limit=50, offset=0) donde q= "genre:rock, year:2020"


# TENEMOS QUE EXTRAER: 
# #250 canciones por género y año (5 años entre 2019-2023), artista (nombre), género musical, tipo (canción = tracks), nombre (canción = "tracks"), año de lanzamiento, (excluimos ID)
# género musical (trance) y tipo (tracks) los añadimos al final porque son parámetros de la búsqueda (no nos van a salir como datos)
# podemos incluir nombre (álbum = "album") como next steps


In [7]:
datos_techno = sp.search(q="genre:trance, year:2019", type="track", limit=50, offset=0)

In [8]:
datos_techno.keys()

dict_keys(['tracks'])

In [9]:
datos_techno["tracks"]["items"][0].keys()
#interesan album, artists, id, name

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'is_playable', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [10]:
datos_techno["tracks"]["items"][0]["name"]
# nombre de la primera cancion

'Exile Babylon'

In [11]:
len(datos_techno["tracks"]["items"]) # tengo 50 items, registros

50

In [12]:
len(datos_techno["tracks"]["items"][0]) # que son 18 keys en items

18

In [13]:
datos_techno["tracks"]["items"][0]["album"].keys()

dict_keys(['album_type', 'artists', 'available_markets', 'external_urls', 'href', 'id', 'images', 'is_playable', 'name', 'release_date', 'release_date_precision', 'total_tracks', 'type', 'uri'])

In [14]:
datos_techno["tracks"]["items"][0]["album"]["release_date"]
# año de lanzamiento (album)

'2019-05-06'

In [15]:
datos_techno["tracks"]["items"][0]["album"]["artists"][0]["name"]
# nombre del artista

'Har-El'

In [16]:
datos_techno["tracks"]["items"][0]["name"]
# nombre de la primera cancion

'Exile Babylon'

In [17]:
datos_techno["tracks"]["items"][0]["album"]["release_date"]
# año de lanzamiento (album)

'2019-05-06'

In [18]:
canciones_techno = {"nombre_artista": [], "nombre_canción": [], "año_lanzamiento": []}

for datos in datos_techno["tracks"]["items"]:
    canciones_techno["nombre_artista"].append(datos["album"]["artists"][0]["name"])
    canciones_techno["nombre_canción"].append(datos["name"])
    canciones_techno["año_lanzamiento"].append(datos["album"]["release_date"])
    
    break  # 1 canción si quitamos el break da las 50 canciones de trance del primer offset (50) del año 2019
canciones_techno

{'nombre_artista': ['Har-El'],
 'nombre_canción': ['Exile Babylon'],
 'año_lanzamiento': ['2019-05-06']}

In [19]:
# parte de búsqueda por año y con el offset: 

for i in range(2019,2020):  # ahora pongo 1 año, uego será (2019,2024)
    print(i)
    for o in range(0,201,50):
        q=f"genre:roca:{i}"
        datos_techno = sp.search(q="genre:trance", type="track",limit=50,offset=o)
        print (datos_techno.keys())
        print(o)
    #for x in range(0,15):
        #(print(datos["tracks"]["items"][x]["name"]))
    #break

2019
dict_keys(['tracks'])
0
dict_keys(['tracks'])
50
dict_keys(['tracks'])
100
dict_keys(['tracks'])
150
dict_keys(['tracks'])
200


In [20]:
import json
import pandas as pd

In [21]:
canciones_techno = {"nombre_artista": [], "nombre_cancion": [], "año_lanzamiento": [], "género_musical": [], "tipo": []}
for anio in range(2019,2024):
    for offset in range(0, 250, 50):
        datos_trance = sp.search(q=f"genre:trance, year:{anio}", type="track", limit=50, offset=offset)
        for cancion in datos_trance["tracks"]["items"]:
            canciones_techno["nombre_artista"].append(cancion["album"]["artists"][0]["name"])
            canciones_techno["nombre_cancion"].append(cancion["name"])
            canciones_techno["género_musical"].append("trance")
            canciones_techno["año_lanzamiento"].append(cancion["album"]["release_date"][0:4]) # para sacar sólo el año
            canciones_techno["tipo"].append(cancion["type"])
df_trance = pd.DataFrame(canciones_techno)
df_trance

,nombre_artista,nombre_cancion,año_lanzamiento,género_musical,tipo
0,Har-El,Exile Babylon,2019,trance,track
1,Stefre Roland,Miami - Original Mix,2019,trance,track
2,Various Artists,Carpo (AIR265),2019,trance,track
3,Various Artists,Cosmodrome - Daniel Kandi Psy'lift Remix,2019,trance,track
4,Various Artists,The Last Conception,2019,trance,track
...,...,...,...,...,...
1245,Bobina,Lazy World - 4 Lazy DJs Mix Remastered,2023,trance,track
1246,Roman Messer Suanda Radio,CYBERIA (Suanda 375),2023,trance,track
1247,Various Artists,Auratoria - Muatoy Remix,2023,trance,track
1248,Gux Jimenez,Dancing in Your Fantasy,2023,trance,track


In [22]:
df_trance.to_csv("technodatabase.csv")

In [23]:
df_values=pd.read_csv("technodatabase.csv",index_col=0)
# ponemos el index_col= 0 para q no nos cree el Unnamed

In [24]:
df_values
# el índice va a salir pero lo quitamos al hacer el itertuples

,nombre_artista,nombre_cancion,año_lanzamiento,género_musical,tipo
0,Har-El,Exile Babylon,2019,trance,track
1,Stefre Roland,Miami - Original Mix,2019,trance,track
2,Various Artists,Carpo (AIR265),2019,trance,track
3,Various Artists,Cosmodrome - Daniel Kandi Psy'lift Remix,2019,trance,track
4,Various Artists,The Last Conception,2019,trance,track
...,...,...,...,...,...
1245,Bobina,Lazy World - 4 Lazy DJs Mix Remastered,2023,trance,track
1246,Roman Messer Suanda Radio,CYBERIA (Suanda 375),2023,trance,track
1247,Various Artists,Auratoria - Muatoy Remix,2023,trance,track
1248,Gux Jimenez,Dancing in Your Fantasy,2023,trance,track


In [25]:
list(df_values.itertuples(index=False, name=None))
# aqui no incluye ni el índice ni los títulos de las colmnas

[('Har-El', 'Exile Babylon', 2019, 'trance', 'track'),
 ('Stefre Roland', 'Miami - Original Mix', 2019, 'trance', 'track'),
 ('Various Artists', 'Carpo (AIR265)', 2019, 'trance', 'track'),
 ('Various Artists',
  "Cosmodrome - Daniel Kandi Psy'lift Remix",
  2019,
  'trance',
  'track'),
 ('Various Artists', 'The Last Conception', 2019, 'trance', 'track'),
 ('Goa Doc',
  'Jess Inspace - Psychedelic Goa Trance 2020 DJ Mixed',
  2019,
  'trance',
  'track'),
 ('Various Artists', 'Morning Dew - Radio Cut', 2019, 'trance', 'track'),
 ('Various Artists', 'Rearm - Extended Mix', 2019, 'trance', 'track'),
 ('Various Artists', 'Lone Wanderer - Original Mix', 2019, 'trance', 'track'),
 ('Airwave',
  'Musings Of A Lifetime - Remastered Album Mix',
  2019,
  'trance',
  'track'),
 ('Various Artists', 'Kill The Fear - Extended Mix', 2019, 'trance', 'track'),
 ('Armin van Buuren ASOT Radio',
  'A State Of Trance (ASOT 941) - Track Recap, Pt. 2',
  2019,
  'trance',
  'track'),
 ('Various Artists', '

In [26]:
values = list(df_values.itertuples(index=False, name=None))

In [27]:
import mysql.connector
from mysql.connector import errorcode

In [28]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='MusicTrends')


mycursor = cnx.cursor()
sql = "INSERT INTO Spotify (nombre_artista, nombre_cancion, año_lanzamiento, género_musical, tipo) VALUES (%s, %s, %s, %s,%s)"
val = values

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

ProgrammingError: 1049 (42000): Unknown database 'musictrends'